In [1]:
import os
import sys

# sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
# os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')
!pip install -r requirements.txt

import PETs_Experiment

In [2]:
loader = PETs_Experiment.Loader(filepath = '../[sunset]/data/[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [3]:
splitter = PETs_Experiment.Splitter(data              = loader.data
                                   ,num_samples       = 30
                                   ,train_split_ratio = 0.8)
print(splitter.data[1]['train'     ].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'     ].head(1) )
print(splitter.data[1]['validation'].head(1) )

39073
9769
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
   age workclass  fnlwgt education  educational-num      marital-status  \
0   27   Private  205145   HS-grad                9  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [4]:
preproc = PETs_Experiment.Preprocessor(
     data = splitter.data[1]['train']
    ,missing=True
        ,missing_method  = 'drop'
        ,missing_columns = None
    ,outlier=True
        ,outlier_method  = 'IQR'
        ,outlier_columns = {'ignore': ['hours-per-week']}
    ,encoding=True
        ,encoding_method  = 'Label'
        ,encoding_columns = None
    ,scaling=True
        ,scaling_method  = 'Standard'
        ,scaling_columns = {'focus' : 'fnlwgt'}
)
print('\n\n\n')
print(preproc.data.head(1))
print('\n\n\n')
print(preproc.encoder)
print('\n\n\n')
print(preproc.scaler)

Preprocessor - Missingist (Drop): Dropped 2895 rows.
Preprocessor - Outlierist (IQR): Dropped  3013 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   233 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped   225 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  1727 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1059 rows on fnlwgt         . Kept [-63222.875, 418198.125] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6047 in 36178 rows.
Preprocessor - Encoder (Label): Column relationship    been labelized from 0 to  5.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column income          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column marital-status 

In [5]:
synthesizer = PETs_Experiment.Synthesizer(data = preproc.data
                                         ,synthesizing_method = 'sdv-singletable-gaussiancoupula'
                                         )
synthesizer.fit_sample()
print(synthesizer.data_syn.head(1))

Synthesizer (SDV - SingleTable): Metafile loading time: 0.0254 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.0482 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30131 rows (same as raw) in 1.6538 sec.
   age  workclass    fnlwgt  education  educational-num  marital-status  \
0   43          2 -0.495437         13               10               0   

   occupation  relationship  race  gender  capital-gain  capital-loss  \
0           4             1     4       0             0             0   

   hours-per-week  native-country  income  
0              43              24       1  


In [6]:
postproc = PETs_Experiment.Postprocessor(data = synthesizer.data_syn
                                        ,encoder = getattr(preproc ,'encoder' ,None)
                                        ,scaler  = getattr(preproc ,'scaler'  ,None)
                                        )
print(postproc.data.head(1))

Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding race.
   age workclass         fnlwgt     education  educational-num marital-status  \
0   43   Private  136087.586752  Some-college               10       Divorced   

        occupation   relationship   race  gender  capital-gain  capital-loss  \
0  Farming-fishing  Not-in-family  White  Female             0             0   

   hours-per-week native-country income  
0              43         Mexico 

In [7]:
para_Executor = {
     'Loader_filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
    ,'Loader_na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
    ,'Splitter_num_samples'       : 2
    ,'Splitter_train_split_ratio' : 0.8
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    ,'Preprocessor_scaling'         : True
    ,'Preprocessor_scaling_method'  : 'Standard'
    ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer_synthesizing_method' : 'sdv-singletable-gaussiancoupula'
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run()


for k ,v in executor.loader.items():
    print(k)
    print(v.data.head(1))

Executor - Loader: [Adt Income] adult.csv loading time: 6.6238 sec.
Executor - Splitter: Default splitting time: 0.0734 sec.
Preprocessor - Missingist (Drop): Dropped 2885 rows.
Preprocessor - Outlierist (IQR): Dropped  3050 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   228 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped   221 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  1722 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1045 rows on fnlwgt         . Kept [-63815.25, 419326.75] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6065 in 36188 rows.
Preprocessor - Encoder (Label): Column relationship    been labelized from 0 to  5.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - 

In [ ]:
para_Executor = {
     'Loader' : {
         'adult'  : {'filepath'   : '../[sunset]/data/[Adt Income] adult.csv'
                    ,'na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
        }
        # ,'NHANES' : {'filepath'      : '../[sunset]/data/[NHANES] B.csv'
        #             ,'header_exist' : False
        #             ,'header_names' : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
        # }
     }
    ,'Splitter' : {
         '0.8'  : {'num_samples'       : 30
                  ,'train_split_ratio' : 0.8
        }
        ,'0.05' : {'num_samples'       : 30
                  ,'train_split_ratio' : 0.05
        }
    }
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    # ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    # ,'Preprocessor_scaling'         : True
    # ,'Preprocessor_scaling_method'  : 'Standard'
    # ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer': {
        'GaussianCoupula' : {'synthesizing_method' : 'sdv-singletable-gaussiancoupula'}
       ,'CoupulaGAN'      : {'synthesizing_method' : 'sdv-singletable-coupulagan'}
       ,'CTGAN'           : {'synthesizing_method' : 'sdv-singletable-ctgan'}
       ,'TVAE'            : {'synthesizing_method' : 'sdv-singletable-tvae'}
    }
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run()
    

Executor - Loader: adult loading time: 6.6902 sec.
Executor - Splitter: 0.8 splitting time: 1.0359 sec.
Preprocessor - Missingist (Drop): Dropped 2876 rows.
Preprocessor - Outlierist (IQR): Dropped  3052 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   228 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped   218 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  9579 rows on hours-per-week . Kept [32.5, 52.5] only.
Preprocessor - Outlierist (IQR): Dropped  1722 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1068 rows on fnlwgt         . Kept [-62030.5, 416181.5] only.
Preprocessor - Outlierist (IQR): Totally Dropped 14084 in 36197 rows.
Preprocessor - Encoder (Label): Column relationship    been labelized from 0 to  5.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (La

Process ForkProcess-136:
Process ForkProcess-95:
Process ForkProcess-99:
Process ForkProcess-133:
Process ForkProcess-116:
Process ForkProcess-104:
Process ForkProcess-86:
Process ForkProcess-90:
Process ForkProcess-97:
Traceback (most recent call last):
Process ForkProcess-132:
Traceback (most recent call last):
Process ForkProcess-129:
Process ForkProcess-121:
Process ForkProcess-101:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkProcess-120:
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process ForkProcess-80:
Process ForkProcess-106:
Process ForkProcess-89:
Traceback (most recent call last):
Process ForkProc

In [1]:
import os
os.cpu_count()

16